In [ ]:
#| default_exp regress_ml

# Train ML model

In [1]:
#|hide
from nbdev.showdoc import *

In [7]:
#|hide
#|export
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from skopt import BayesSearchCV
from ML_projects import const_vals as CONST

from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor

import warnings
warnings.filterwarnings('ignore')

In [8]:
#|export 
class TrainRegression():
       
       def __init__(self,
               df_path : str , # path to dataframe to be used to train. File should be CSV file
               requested_model : str , # model to train. Options : 'RFR' 'XGB' 'SVR' 'RIDGE' 'KNEIGHBORS' 'GRADIENT_BOOST' 'ADA'
               ground_truth_col: str, # name of the column with true data to train
               test_size : float , #size of data to be used for test 
               hyper_method : str , #hyperparameter tunning method. accepts : 'randomized' 'bayesian' , 'bayesian continous'
               columns_to_remove : list[str]=None , #columns not to use for trainning the model. These columns will be removed.
              #  hyper_params : dict = CONST.RANDOM_GRID_RFR, #parameters for hyperparameter tunning
              #  space : list = CONST.SPACE_RFR  , #
               ):
             self.df_path = df_path
             self.columns_to_remove = columns_to_remove
             self.ground_truth_col = ground_truth_col
             self.test_size = test_size
             self.hyper_method = hyper_method
             self.model_str = requested_model

             #load data and get train test data
             self.x_train, self.x_test, self.y_train, self.y_test = self._load_df_split_data()
             
             # create initial model and match the params 

             self.model , self.params = self._match_models_()

             #hyperparameter tunning
             self.best_params = self.hyperparameter()

             #train best model 

             self.best_model = self._create_best_model()

             #fit model

             self.best_model.fit(self.x_train, self.y_train)


       

       def _match_models_(self):
            self.model = CONST.algorithm_to_model[self.model_str]
            self.params = CONST.algorithm_to_params[self.model_str]

            return self.model , self.params

       def _load_df_split_data(self):
               
               self.df = pd.read_csv(self.df_path)
               #load dataframe
               if self.columns_to_remove!= None:
                     self.df = self.df.drop(self.columns_to_remove,axis=1)
          
               # split to x,y and train test data
               self.x = self.df.drop(self.ground_truth_col,axis=1)
               self.y = self.df[self.ground_truth_col].values

               #split data to train and test
               self.x_train, self.x_test, self.y_train, self.y_test = train_test_split(
                      self.x, self.y, test_size=self.test_size, random_state=42)

               return self.x_train, self.x_test, self.y_train, self.y_test
       

       def hyperparameter(self):
             if self.hyper_method == 'randomized':
                   rf_random = RandomizedSearchCV(estimator = self.model, 
                                                  param_distributions = self.params,
                                                  n_iter = CONST.N_ITERATIONS_RFR,
                                                  cv = CONST.CV_RFR, 
                                                  verbose=CONST.VERBOSE , 
                                                  random_state=CONST.RANDOM_STATE , 
                                                  n_jobs = CONST.N_JOBS)
                   
                   #fit model 
                   rf_random.fit(self.x_train, self.y_train)

                  #best params
                   self.best_params = rf_random.best_params_


             elif self.hyper_method == 'bayesian':
                  rf_bayes = BayesSearchCV(self.model,
                                           search_spaces=self.params, 
                                           n_iter=CONST.N_ITERATIONS_RFR, 
                                           cv=CONST.CV_RFR)
                  np.int = int
      
                  #fit model 
                  rf_bayes.fit(self.x_train, self.y_train)

                  self.best_params = rf_bayes.best_params_

                  print(f'best params : {rf_bayes.best_params_}')


             return self.best_params
       
       def _create_best_model(self):
             if self.model_str == 'RFR':
                   self.best_model = RandomForestRegressor(**self.best_params)
            
             elif self.model_str == 'XGB':
                   self.best_model =xgb.XGBRegressor(**self.best_params)
             
             elif self.model_str == 'SVR':
                   self.best_model = SVR(**self.best_params)

             elif self.model_str == 'RIDGE':
                   self.best_model = Ridge(**self.best_params)
            
             elif self.model_str == 'KNEIGHBORS':
                   self.best_model = KNeighborsRegressor(**self.best_params)

             elif self.model_str == 'GRADIENT_BOOST':
                   self.best_model = GradientBoostingRegressor(**self.best_params)

             elif self.model_str == 'ADA':
                   self.best_model = AdaBoostRegressor(**self.best_params)

             return self.best_model
       
       def evaluate_model(self):
             print('not written yet')

                   
              





In [9]:
instance = TrainRegression(
  df_path=r"D:\git\ML_projects\nbs\data\resampled_sen2.csv",
  requested_model= 'RFR',
  ground_truth_col = "TOC",
  test_size = 0.25,
  columns_to_remove = ['Unnamed: 0.1', 'Unnamed: 0', 'Lon', 'Lat', 'clay', 'silt','sand', 'NI'],
  hyper_method = 'bayesian'    
  )


# test = instance.the_best_model
# print(instance.the_best_params)


AttributeError: 'TrainRegression' object has no attribute 'hyper_params'

In [ ]:
print(test)